Import packages and library

In [43]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
import os


Mount my Google Drive to Colab 

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
%cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [46]:
%ls

allImageFolder/        test/            train2/           Visualize.ipynb
F2_score.ipynb         test2/           train.csv
ObjectDetection.ipynb  test_labels.csv  train_labels.csv
Preprocessing.ipynb    train/           val_labels.csv


In [47]:
!apt install detecto

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package detecto


In [48]:
!pip3 install detecto


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_320_fpn
from torchvision.models.mobilenetv3 import MobileNetV3
from detecto.core import Model, Dataset, DataLoader
x_train = Dataset('train_labels.csv','train')
validation_set = Dataset('val_labels.csv','validation')
# loader = DataLoader(x_train, batch_size=2, shuffle=True)
model = Model(["starfish"])
model.fit(x_train, learning_rate = 0.003, epochs=5)
# losses = model.fit(loader, validation_set, epochs = 5)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1 of 5
Begin iterating over training dataset


100%|██████████| 160/160 [04:15<00:00,  1.60s/it]


Epoch 2 of 5
Begin iterating over training dataset


100%|██████████| 160/160 [00:47<00:00,  3.37it/s]


...

100%|██████████| 160/160 [00:47<00:00,  3.40it/s]


Epoch 4 of 5
Begin iterating over training dataset


100%|██████████| 160/160 [00:47<00:00,  3.39it/s]


Epoch 5 of 5
Begin iterating over training dataset


100%|██████████| 160/160 [00:47<00:00,  3.40it/s]


In [50]:
from pandas.core.internals.blocks import soft_convert_objects
from detecto.utils import read_image
from detecto.visualize import show_labeled_image
from PIL import Image
img_list = []
predictImagePathList = os.listdir('test2')
print(predictImagePathList)
for i in range(len(predictImagePathList)):
  img_list.append(Image.open('test2/' + predictImagePathList[i]))
# print(predictImagePathList)
prediction = model.predict(img_list)
# prediction = model.predict(predictImage)

['0-5018.jpg', '0-4568.jpg', '0-9879.jpg', '0-4475.jpg', '0-4473.jpg']


In [51]:
print(len(prediction))

5


In [52]:
test_csv_df = pd.read_csv('test_labels.csv')

In [53]:
labels = [sublist[0] for sublist in prediction]
boxes = [list(np.array(sublist[1]).astype(int)) for sublist in prediction]
for i in range(len(boxes)):
  for j in range(len(boxes[i])):
    boxes[i][j] = list(boxes[i][j] )
scores = [list(np.array(sublist[2])) for sublist in prediction]

test_csv_df = pd.read_csv('test_labels.csv')
test_csv_df['labels'] = labels
test_csv_df['boxes'] = boxes
test_csv_df['scores'] = scores


In [54]:
test_csv_df

,filename,width,height,class,xmin,ymin,xmax,ymax,image_id,labels,boxes,scores
0,2-6385.jpg,1280,720,starfish,108,313,141,346,0,"[starfish, starfish, starfish, starfish, starf...","[[430, 546, 497, 598], [343, 249, 407, 308], [...","[0.6159917, 0.12308445, 0.06006297, 0.05301181..."
1,2-5446.jpg,1280,720,starfish,116,439,175,494,1,"[starfish, starfish, starfish, starfish, starf...","[[968, 218, 1032, 277], [283, 644, 334, 689], ...","[0.53453386, 0.3368647, 0.18505572, 0.15932739..."
2,2-5760.jpg,1280,720,starfish,578,336,642,388,2,"[starfish, starfish, starfish, starfish, starf...","[[346, 5, 415, 69], [1150, 295, 1212, 346], [1...","[0.90046465, 0.5777868, 0.5104897, 0.41638777,..."
3,1-9274.jpg,1280,720,starfish,343,221,386,248,3,"[starfish, starfish, starfish, starfish, starf...","[[586, 2, 621, 30], [341, 545, 407, 599], [104...","[0.1992421, 0.1780093, 0.11942416, 0.11097678,..."
4,2-7990.jpg,1280,720,starfish,416,486,452,520,4,"[starfish, starfish, starfish, starfish, starf...","[[677, 465, 738, 513], [617, 493, 674, 553], [...","[0.8559387, 0.7534933, 0.65773207, 0.46527225,..."


In [55]:
test_csv_df['ground_truth'] = test_csv_df.apply(lambda row: [int(row.xmin), int(row.ymin), int(row.xmax), int(row.ymax)],axis=1)

In [56]:
test_csv_df

,filename,width,height,class,xmin,ymin,xmax,ymax,image_id,labels,boxes,scores,ground_truth
0,2-6385.jpg,1280,720,starfish,108,313,141,346,0,"[starfish, starfish, starfish, starfish, starf...","[[430, 546, 497, 598], [343, 249, 407, 308], [...","[0.6159917, 0.12308445, 0.06006297, 0.05301181...","[108, 313, 141, 346]"
1,2-5446.jpg,1280,720,starfish,116,439,175,494,1,"[starfish, starfish, starfish, starfish, starf...","[[968, 218, 1032, 277], [283, 644, 334, 689], ...","[0.53453386, 0.3368647, 0.18505572, 0.15932739...","[116, 439, 175, 494]"
2,2-5760.jpg,1280,720,starfish,578,336,642,388,2,"[starfish, starfish, starfish, starfish, starf...","[[346, 5, 415, 69], [1150, 295, 1212, 346], [1...","[0.90046465, 0.5777868, 0.5104897, 0.41638777,...","[578, 336, 642, 388]"
3,1-9274.jpg,1280,720,starfish,343,221,386,248,3,"[starfish, starfish, starfish, starfish, starf...","[[586, 2, 621, 30], [341, 545, 407, 599], [104...","[0.1992421, 0.1780093, 0.11942416, 0.11097678,...","[343, 221, 386, 248]"
4,2-7990.jpg,1280,720,starfish,416,486,452,520,4,"[starfish, starfish, starfish, starfish, starf...","[[677, 465, 738, 513], [617, 493, 674, 553], [...","[0.8559387, 0.7534933, 0.65773207, 0.46527225,...","[416, 486, 452, 520]"


In [57]:
test_csv_df.to_csv('test_prediction004.csv', index=False)